# Homework 1
## 36-615, Spring 2023

TODO YOUR NAME HERE

In [ ]:
import torch
import torch.distributions
import torch.nn as nn

import matplotlib.pyplot as plt
import numpy as np

Because of the timing, this homework isn't super computational intensive. Instead it's meant to force you to get PyTorch working and start getting familiar with how it works. Be sure to open the [PyTorch documentation](https://pytorch.org/docs/stable/index.html) and start learning how to find things you need.

## Problem 1: Gradient Descent

Let's consider implementing gradient descent on a problem we can easily optimize analytically: linear regression. This is technically pointless since you have an analytical solution. But you already understand linear regression, so it'll be easier to understand gradient descent and its pitfalls.

To make things slightly more interesting, let's do *weighted* linear regression. In weighted linear regression, we solve the optimization problem
$$
\begin{align*}
\hat \beta &= \operatorname*{argmin}_\beta \text{RSS} \\
&= \operatorname*{argmin}_\beta \sum_{i=1}^n w_i (Y_i - \hat Y_i)^2.
\end{align*}
$$

Here $w$ is a vector of weights, such that $w_i \geq 0$ for all $i$, that are chosen or known in advance. For example, if we know the responses have unequal variance and know the variances, setting $w_i \propto 1/\operatorname{var}(\epsilon_i)$ will allow us to account for the unequal variances.

In this problem, we'll have two covariates and an intercept, so $\hat Y_i = \hat \beta_0 + \hat \beta_1 X_1 + \hat \beta_2 X_2$. The covariates are correlated with each other. Here are the true betas:

In [ ]:
beta_0 = torch.tensor(7.)
beta = torch.tensor([-2., 2.])

We'll simulate 100 observations of data with normally distributed error.

In [ ]:
N = 100

X = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(2),
                                                               torch.tensor([[1, 0.7], [0.7, 1]])).sample((N,))
Y = beta_0 + X @ beta + torch.normal(mean=torch.zeros(N), std=torch.tensor([1.]))
w = torch.rand(N)

### Problem 1 (a): The Shape of the Objective

If we fix the intercept at its true value and vary the slopes, we can plot the objective function against the two slopes using a contour plot. 

Fill in the code below to plot the RSS when $\beta_0 = 7$ and we change $\beta_1$ and $\beta_2$ across the range given.

*Hint:* The [`.item()` method](https://pytorch.org/docs/stable/generated/torch.Tensor.item.html#torch.Tensor.item) can convert a `Tensor` containing a single value into an ordinary Python number.

In [ ]:
b1s = np.linspace(-8, 4)
b2s = np.linspace(-4, 8)

b1, b2 = np.meshgrid(b1s, b2s)

RSS_grid = np.zeros_like(b1)

for ii in range(b1.shape[0]):
    for jj in range(b1.shape[1]):
        beta = torch.tensor([b1[ii, jj], b2[ii, jj]], dtype=torch.float)
        # TODO fill in:
        RSS_grid[ii, jj] = 
        
plt.contour(b1, b2, RSS_grid, levels=20)
plt.xlabel("beta1")
plt.ylabel("beta2")

Comment on the shape of this plot. What feature of the data causes its shape?

### Problem 1 (b): Implementing Gradient Descent

The function `regression_gradient_descent()` implements gradient descent for linear regression. Given starting values for the intercept and slopes, it calculates the residual sum of squares, uses PyTorch to get its gradients, and updates the intercept and slopes accordingly.

Fill in the function where indicated.

In [ ]:
def regression_gradient_descent(beta_0, beta, max_iter=100, learning_rate=0.001):
    """
    Run gradient descent on the linear regression model.
    
    - beta_0: Scalar giving the starting value for the intercept.
    - beta: List of two entries, giving the starting values for the slopes.
    - max_iter: Maximum number of iterations allowed.
    - learning_rate: Learning rate used.
    
    Returns a (betas, errors). betas is a numpy array of beta values with two 
    columns, one per slope; errors is a list of RSS values at each iteration.
    """

    errors = []
    betas = [beta]
    prev_error = float("+inf")

    # Starting values
    beta_0 = torch.tensor(beta_0, requires_grad=True, dtype=torch.float)
    beta = torch.tensor(beta, requires_grad=True, dtype=torch.float)

    for ii in range(max_iter):
        # Calculate RSS and store the value
        RSS = # TODO fill in
    
        new_error = RSS.item()
    
        errors.append(new_error)
    
        # check if we have converged
        if abs(new_error - prev_error) / prev_error < 0.0001:
            break
        prev_error = new_error
    
        # back-propagate to get gradients
        RSS.backward()
    
        # Don't track gradients of these calculations -- otherwise PyTorch would
        # try to take gradients of this math too
        with torch.no_grad():
            # hint: the .data property of tensors can be used to access their numerical contents
            beta -= # TODO fill in
            beta_0 -= # TODO fill in
        
            # Store this iteration's beta estimate
            betas.append(beta.tolist())
        
            # Reset the gradient information to zero
            beta.grad.zero_()
            beta_0.grad.zero_()

    return np.array(betas), errors

To test your function, run the following code. It runs gradient descent with particular starting values, then plots the $\beta$ values from each iteration. The true values are marked in red.

In [ ]:
betas, errors = regression_gradient_descent(0., [-7., 0.])

plt.contour(b1, b2, RSS_grid, levels=20)
plt.scatter([-2.], [2.], color="red") # the true minimum
plt.plot(betas[:, 0], betas[:, 1], '-o') # from  gradient descent
plt.xlabel("beta1")
plt.ylabel("beta2")

It's also common to plot the error over each iteration; the error should generally decrease over time.

In [ ]:
plt.plot(range(len(errors)), errors)
plt.xlabel("Iteration")
plt.ylabel("RSS");

### Problem 1 (c): The Dangers of Gradient Descent

Above, you used the default learning rate of $\gamma = 0.001$. In vanilla gradient descent, the learning rate is a key parameter.

Plot the solution path (the sequence of $\beta$ values) on top of the contour plot, as you did in part (b), for three learning rates. Choose one learning rate so the fit converges extremely slowly, one so it converges quickly, and one so that it does not converge at all, no matter how many iterations you allow it to take.

Also, for each value, plot the RSS versus iteration, as you did in part (b). Comment on the *speed* of convergence for each fit, meaning the number of iterations taken to reach a particular value of RSS.

### Problem 1 (d): Using PyTorch Modules

PyTorch has the idea of building *modules*. Each module is made out of layers, where a layer takes inputs, does some calculation, and produces outputs; the outputs of one layer become the inputs of another. We'll see this in action when we do neural networks, which are composed of many layers.

In a neural network, a *linear layer* does the transformation $y = xA^T + b$, where $x \in \mathbb{R}^p$, $A \in \mathbb{R}^{q \times p}$, $b \in \mathbb{R}^q$, and $y \in \mathbb{R}^q$. In neural network terms, $A$ is a matrix of *weights* and $b$ is a vector of *biases*. Note that we could have $q \neq p$, so a linear layer takes vectors and converts them to vectors of potentially different dimension.

If we want to represent a linear regression model that takes two covariates and an intercept and produces a scalar output---like the data we simulated---what values of $q$ and $p$ should we use?

$q = ?$, $p = ?$

The [`torch.nn.Linear()` class](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) represents a linear layer. Figure out the correct arguments to create the linear layer we want. (We imported `torch.nn` as `nn`, so you can just use `nn.Linear`.)

In [ ]:
nn.Linear(TODO)

We then create an object with a bit of boilerplate. This object represents our network and does the calculations. The `forward()` method specifies how to do the forward pass of the calculations---how to take `x` and calculate `y`, in our case.

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        
        self.linear = # TODO fill in your linear layer
        
    def forward(self, x):
        return self.linear(x)

Next, we create an instance of the class and create an optimizer. The `SGD` optimizer does stochastic gradient descent (we'll explain "stochastic" soon). It does much of the work you did in (b) automatically.

In [ ]:
model = LinearRegression()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

Now we train the model. This is a template for how model training will look for many kinds of models in PyTorch.

In [ ]:
def train(model, optimizer, max_iter=10000):
    # Storage
    losses = []
    
    for ii in range(max_iter):
        # Reset all gradients
        optimizer.zero_grad()
    
        # Obtain the model's predictions with the data
        predictions = model(X)
    
        # Get the loss, using the known Ys
        loss = nn.functional.mse_loss(predictions[:, 0], Y)
    
        # Calculate the gradients with backpropagation
        loss.backward()
    
        # Perform one gradient descent step
        optimizer.step()

In [ ]:
train(model, optimizer)

Here we print out the parameter estimates:

In [ ]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Interpret these estimates. There is a weight and a bias; write out the fit in the form of the estimated regression equation:

$y = ? + ? x_1 + ? x_2$